In [ ]:
# Importing libraries

%matplotlib inline

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [ ]:
## Normalize data
# uncomment next two lines to findout the shape of train and test set
# x_train.shape  
# x_test.shape

x_train = np.reshape(x_train, (60000, 28*28))
x_test = np.reshape(x_test, (10000, 28*28))
#Normalization
x_train = x_train/255.
x_test = x_test/255.

In [ ]:
#Creating Neural Network

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation = 'sigmoid', input_shape=(784,)),
    tf.keras.layers.Dense(32, activation = 'sigmoid'),
    tf.keras.layers.Dense(32, activation = 'softmax')
])
#we could have used ReLU but it will give positive values only. Or else we could use ReLU and then normalized it
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
#Training the model
_ = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=20, batch_size=1024,
    verbose=2
)

In [ ]:
model.save('model.h5')

In [ ]:
%%writefile ml_server.py

import json
import tensorflow as tf
import numpy as np
# import random

from flask import Flask, request

app = Flask(__name__)

model = tf.keras.models.load_model('model.h5')
feature_model = tf.keras.models.Model(
    model.inputs,
    [layer.output for layer in model.layers]
)

_, (x_test, _) = tf.keras.datasets.mnist.load_data()
x_test = x_test / 255.

def get_prediction():
    index = np.random.choice(x_test.shape[0])
    image = x_test[index, :, :]
    image_arr = np.reshape(image, (1,784))
    return feature_model.predict(image_arr), image

@app.route('/', methods=['GET','POST'])
def index():
    if request.method == 'POST':
        preds, image = get_prediction()
        final_preds = [p.tolist() for p in preds]
        return json.dumps({
            'prediction': final_preds,
            'image': image.tolist()
        })
    return 'Welcome to my page! - Supriya Anand'

if __name__ == '__main__':
    app.run()

In [ ]:
%%writefile app.py

import streamlit as st      #a package to run web app
import json
import requests
import matplotlib.pyplot as plt
import numpy as np

URI = 'http://127.0.0.1:5000'

st.title('Neural Network Web Visualizer')
st.sidebar.markdown('##input image')

if st.button('Get random prediction'): 
    response = requests.post(URI, data={})
    response = json.loads(response, text)
    preds = response.get('prediction')
    image = response.get('image')
    image = np.reshape(image, (28,28))
    
    st.sidebar.image(image, width=150)
    
    for layer, p in enumerate(preds):
        numbers = np.squeeze(np.array(p))
        plt.figure(figsize=(32,4))
        
        if layer == 2:
            row = 1
            col = 10
        else:
            row = 2
            column = 16
        
        for i, number in em=numerate(numbers):
            plt.subplot(row, col, i+1)
            plt.imshow(number*np.ones(8,8,3).astype('float32'))
            plt.xticks([])
            plt.ytick([])
            
            if layer == 2:
                plt.label(str(i), fontsize = 40)
            
            plt.subplots_adjust(wspace=0.05, hspace = 0.05)
            plt.tight_layout()
            st.text('Layer {}'.format(layer+1))
            st.pyplot()
        